In [ ]:
!pip install missingno

In [ ]:
import pandas as pd
import missingno as msno
data=pd.read_csv('titanic.csv')
data.info()

In [ ]:
msno.matrix(data)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(data=data,x='Age',kde=True)
plt.show()

In [ ]:
data.describe(include='all')

In [ ]:
data.drop(['PassengerId','Name','Ticket','Fare'],axis=1,inplace=True)
data.columns

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
sm=SimpleImputer(strategy='median') 
le=LabelEncoder()
data['Embarked']=le.fit_transform(data[['Embarked']])
si=SimpleImputer(strategy='most_frequent')data.head()
data['Age']=sm.fit_transform(data[['Age']])
data['Embarked']=si.fit_transform(data[['Embarked']])

data['Sex']=le.fit_transform(data[['Sex']])

In [ ]:
ss=SimpleImputer(strategy='most_frequent')
data['Cabin']=le.fit_transform(data[['Cabin']])
data['Cabin']=ss.fit_transform(data[['Cabin']])

In [ ]:
data.head()

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2,mutual_info_classif
X=data.drop('Survived',axis=1)
Y=data['Survived']

In [ ]:
skbest=SelectKBest(chi2,k='all').fit(X,Y).scores_
minfo=mutual_info_classif(X,Y)
missing_value_ratio=X.isnull().sum()/len(X)
feature_selection=pd.DataFrame({
    'Features':X.columns,
    'chi2':skbest,
    'minfo':minfo,
    'missing_ratio':missing_value_ratio
})
feature_selection

In [ ]:
!pip install skfeature-chappers

In [ ]:
from skfeature.function.similarity_based import fisher_score
import numpy as np
fscore=fisher_score.fisher_score(X.to_numpy(),Y.to_numpy())
feature_fisherscore=sorted(zip(X.columns,fscore),key=lambda x:x[1],reverse=True)
feature_fisherscore

In [ ]:
!pip install mlxtend

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.metrics import accuracy_score

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,
                                               random_state=42)

In [ ]:
log_reg=LogisticRegression(max_iter=1000,random_state=42)

In [ ]:
skf=SFS(estimator=log_reg,n_features_to_select='auto',
       direction='forward',scoring='accuracy',cv=5)

In [ ]:
skf=skf.fit(X_train,Y_train)

In [ ]:
skf.n_features_to_select_

In [ ]:
print(X_train.columns)
print(skf.support_)

In [ ]:
print(skf.n_features_in_)

In [ ]:
efs=EFS(estimator=log_reg,min_features=1,
       max_features=X_train.shape[1],
       scoring='accuracy',
       cv=5)
efs=efs.fit(X_train,Y_train)

In [ ]:
efs.best_feature_names_

In [ ]:
efs.best_score_

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs=SFS(estimator=log_reg,
                             k_features='best',
                             forward=True,
                             floating=False,
                             scoring='accuracy',
                             cv=5)
sfs=sfs.fit(X_train,Y_train)
sfs.k_feature_names_